In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama3-70b-8192",
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
)

d:\anaconda\envs\llmapp\Lib\site-packages\langchain_groq\chat_models.py:355: UserWarning: WARNING! max_completion_tokens is not default parameter.
                    max_completion_tokens was transferred to model_kwargs.
                    Please confirm that max_completion_tokens is what you intended.
  warnings.warn(
d:\anaconda\envs\llmapp\Lib\site-packages\langchain_groq\chat_models.py:355: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


RunnablePassthrough


In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [4]:
result = chain.invoke("Hello, world!")
print(result)

Hello, world!


RunnableLambda


In [5]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [6]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

result = chain.invoke("Ivan")
print(result)

Ivanovich


RunnableParallel


In [7]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [9]:
chain.invoke("Barack Obama")

{'operation_a': 'Barack Obama', 'operation_b': 'Barack Obamaovich'}

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [13]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [14]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model | output_parser

In [15]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

"Here's one:\n\nAbramovich, the Russian billionaire and former owner of Chelsea FC, has a rather unusual hobby - he's a passionate collector of airships! Yes, you read that right. Abramovich has a fascination with dirigibles, those large, lighter-than-air aircraft that were popular in the early 20th century.\n\nIn fact, he reportedly owns several vintage airships, including a 1920s-era Zeppelin and a 1930s-era hydrogen-filled LZ-129 Hindenburg (yes, that Hindenburg, the one that famously caught fire in 1937).\n\nAbramovich has even been known to host lavish parties on his airships, complete with champagne and caviar. Who knew that beneath the tough, business-like exterior of a billionaire oligarch lay a romantic nostalgia for airships?\n\nI hope you found that curious fact about Abramovich interesting!"

Let's see a more advanced use of RunnableParallel


In [16]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_10292\1849545125.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
d:\anaconda\envs\llmapp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("data/Razib.txt")
documents = loader.load()
load_data=documents[0].page_content
vectorstore = FAISS.from_texts(
    [load_data], embeddings, metadatas=[{"source": "Razib.txt"}]
)

In [21]:
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
chain = RunnableParallel(
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
) | prompt | model | StrOutputParser()
result = chain.invoke("What is the name of the author of the text?")
print(result)

The name of the author of the text is Razib Dash.


In [26]:
from operator import itemgetter
vectorstore = FAISS.from_texts(
    [load_data], embeddings, metadatas=[{"source": "Razib.txt"}]
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain=(
    {
     "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
}| prompt | model | StrOutputParser()
)

chain.invoke({
    "question": "What is the name of the author of the text?",
    "language": "hindi"
})


'लेखक का नाम रज़िब डैश (Razib Dash) है.'

In [28]:
chain.invoke({
    "question": "What is Razib?",
    "language": "bangla"
})

'রাজিব একজন শিক্ষাবিদ এবং প্রযুক্তিবিদ।'